In [1]:
import requests
import json
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from database_utils import Database
import pandas as pd
import time
import datetime
import random
import sys

In [ ]:
def LoadUserAgents(uafile):
    uas = []
    with open(uafile, 'rb') as uaf:
        for ua in uaf.readlines():
            if ua:
                uas.append(ua.strip()[:-1])
    random.shuffle(uas)
    return uas

uas = LoadUserAgents("user_agents.txt")

# 1. 根据关键词查出所有视频的相关信息

In [ ]:
keyword="鸿蒙系统"
res=[]

In [ ]:
# 按url获取当前页的视频列表
def get_video_list(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    all_videos = soup.find_all(name='ul',attrs={"class":"video-contain clearfix"})[0]
    all_videos = all_videos.find_all(name='div', attrs={"class":"info"})
    return all_videos
# 从url中分离出up主id
def get_up_id(url):
    return url[url.index("com/") + 4: url.index("?")]
# 获取视频中的信息
def get_vider_data(source, res):
    for i, every_video in enumerate(source):
        data = {}
        data['av_id'] = every_video.find_all(name='span', attrs={"class":"type avid"})[0].text[2:]
        data['title'] = every_video.a['title']

        tags = every_video.find_all(name='div', attrs={"class":"tags"})[0]
        for j, s in enumerate(tags):
            data[s['title']] = s.text.strip()
            if j == 3:
                data['up_id'] = get_up_id(s.a['href'])
        res.append(data)

In [ ]:
for page in range(1, 100):
    url = r'https://search.bilibili.com/all?keyword=' + keyword +'&page=' + str(page)
    video_list = get_video_list(url)
    if len(video_list) == 0:
        print("共%d页"% (page - 1))
        break
    else:
        get_vider_data(video_list, res)   

video_info = pd.DataFrame(data=res)

### 对视频信息进行补充

In [ ]:
def get_video_other_info(row):
    time.sleep(0.7)
    oid = row['av_id']
    print(oid)
    head = {
    'User-Agent': random.choice(uas),
    'Connection': 'close'
    } 
    requests.adapters.DEFAULT_RETRIES =5
    curUrl = 'https://api.bilibili.com/x/web-interface/archive/stat?aid={}'.format(oid)
    source = requests.get(curUrl, headers=head, timeout=3000).text       
    source = json.loads(source)['data']
    row['view']     = source['view']
    row['reply']    = source['reply']
    row['favorite'] = source['favorite']
    row['coin']     = source['coin']
    row['share']    = source['share']
    row['like']     = source['like']
    return row

In [ ]:
video_info = video_info.apply(get_video_other_info, axis=1)

In [ ]:
video_info.sort_values('like', ascending=False)

In [ ]:
video_info.to_csv("video_info.csv", index=False)

# 2. 爬取所有视频的弹幕

In [ ]:
barrages = []
av_id_list = video_info[video_info['弹幕'] != '0']['av_id'].tolist()

In [ ]:
def get_cid(avid):
    source = requests.get("https://api.bilibili.com/x/web-interface/view?aid=" + str(avid)).text
    source = json.loads(source)
    cid = source['data']['cid']
    return cid

In [ ]:
cols_name = ['video_time', 'barrage_type', 'font_size', 'font_color',
             'time', 'barrage_pool_type', 'sender_id', 'barrage_id']
def get_barrages(avid):
    curr_list = []
    cid = get_cid(avid)
    source = requests.get("https://comment.bilibili.com/"+ str(cid) +".xml")
    source.encoding = 'utf-8'
    source = BeautifulSoup(source.text, 'html.parser')
    source = source.find_all(name='d')
    for line in source:
        data = {'av_id': avid, 'content': line.text}
        for i, val in enumerate(line['p'].split(',')):
            data[cols_name[i]] = val
#     source = json.loads(source)
        curr_list.append(data)
    return curr_list

In [ ]:
for i, avid in enumerate(av_id_list):
    barrages += get_barrages(avid)

In [ ]:
barrages_csv = pd.DataFrame(data=barrages)
barrages_csv.to_csv("barrage_csv.csv", index=False)

In [ ]:
barrages_csv

### 添加弹幕中的所有u_id

In [ ]:
def get_uid(x):
    res = database_util.get_uid(x)
    return res


In [ ]:
database_util = Database()
barrages_csv['u_id'] = barrages_csv['sender_id'].apply(lambda x: get_uid(x))
database_util.close()

In [ ]:
barrages_csv.to_csv("barrage_csv.csv", index=False)

# 3. 获取弹幕中所有用户的信息

In [ ]:
barrages_csv = pd.read_csv("barrage_csv.csv")

In [ ]:
barrages_csv = barrages_csv[barrages_csv['u_id'].notnull()]
barrages_csv['u_id'] = barrages_csv['u_id'].astype(int).astype(str)

In [ ]:
def get_user_info(uid):
    
    head = {
    'User-Agent': random.choice(uas),
    'Connection': 'close',
    }
    data = {'u_id': uid}
    requests.adapters.DEFAULT_RETRIES =5
    source = requests.get("https://api.bilibili.com/x/space/acc/info?mid="+ str(uid) +"&jsonp=jsonp",\
                          timeout=3000)
    res = requests.get(
                'https://api.bilibili.com/x/relation/stat?vmid=' + str(uid) + '&jsonp=jsonp').text
    js_fans_data = json.loads(res)         
    source = json.loads(source.text)['data']
    data['name'] = source['name']
    data['sex'] = source['sex']
    data['sign'] = source['sign']
    data['level'] = source['level']
    data['birthday'] = source['birthday']
    data['vip'] = 1 if source['vip']['status'] == 1 else 0 
    data['following'] = js_fans_data['data']['following']
    data['fans'] = js_fans_data['data']['follower']
    return data

In [ ]:
user_info_list=[]

In [ ]:
user_list = barrages_csv['u_id'].tolist()
idx = -1
while idx < len(user_list):
    idx += 1
    
    if idx == len(user_list):
        break
        
    try:
        user_info_list.append(get_user_info(user_list[idx]))
    except:
        print("Connection refused by the server..and i is" + str(idx))
        time.sleep(5)
        idx -= 1
        continue

In [ ]:
user_info = pd.DataFrame(data=user_info_list).drop_duplicates('u_id')
user_info.to_csv('user_info.csv', index=False)

# 获取视频评论信息

In [70]:
from bs4 import BeautifulSoup
import math
def getPageCount(oid):
    headers={
        'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
    }
    baseurl = "https://api.bilibili.com/x/v2/reply?type=1&oid="+str(oid)+"&pn="+str(1)
    r_text = requests.get(baseurl,headers).text
    response = json.loads(r_text)
    if 'data' in response:
        count = response['data']['page']['count']
        pages = math.ceil(count/20)
    else:
        print(oid)
        pages = -1
    return pages

def fetchurl(oid):
    time.sleep(0.7)
    headers={
        'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
    }
    r_text=[]
    page_count = getPageCount(oid)
    if page_count == -1:
        return None
    else:
        for pn in range(1,page_count+1):
            baseurl = "https://api.bilibili.com/x/v2/reply?type=1&oid="+str(oid)+"&pn="+str(pn)
            r_text.append(requests.get(baseurl,headers).text)  
    return r_text  

def parse_text(html_text):  
    if html_text is None:
        return []
    res = []
    
    for text in html_text:
        response = json.loads(text)
        for i in range(len(response['data']['replies'])):
            replies = response['data']['replies'][i]
            
            data ={}
            replies = response['data']['replies'][i]
            data['u_id'] =  replies['mid']
            data['rpid'] = replies['rpid']
            data['oid'] = replies['oid']
            data['ctime'] = replies['ctime']
            data['review'] = replies['content']['message']
            review_count = int(replies['rcount'])
            data['parent_comment_id'] = 0
            res.append(data)
            source = response['data']['replies'][i]['replies']
            if source is not None and len(source) != 0:
                for j in range(len(response['data']['replies'][i]['replies'])):
                    data ={}
                    replies = response['data']['replies'][i]['replies'][j]
                    data['rpid'] = replies['rpid']
                    data['oid'] = replies['oid']
                    data['ctime'] = replies['ctime']
                    data['review'] = replies['content']['message']
                    data['parent_comment_id'] = replies['parent_str']
                    data['u_id'] = replies['mid']
                    res.append(data)
    return res

In [71]:
video_info = pd.read_csv('video_info.csv')
oids = video_info['av_id'].tolist()
oids = list(map(str, oids))

In [72]:
comments_data = []
idx = -1
while idx < len(oids):
    idx += 1
    
    if idx == len(oids):
        break  
    try:
        curr_list = parse_text(fetchurl(oids[idx]))
        if len(curr_list) > 0:
            comments_data += curr_list
    except:
        print("Connection refused by the server..and i is" + str(idx))
        time.sleep(5)
        idx -= 1
        continue

Connection refused by the server..and i is8
54970383


In [73]:
comment_info = pd.DataFrame(data=comments_data)
comment_info.to_csv('comment_info.csv', index=False)